In [1]:
import json
import os

timestamps = [i * 0.02 for i in range(1500 + 1)]
replace = {
    'Aee.': 'A.',
    'Bee.': 'B.',
    'See.': 'C.',
    'Dee.': 'D.',
}

In [2]:
with open('mallm.json') as fopen:
    d = json.load(fopen)
    
len(d)

6564

In [3]:
result = []

for i in range(len(d)):
    audio = os.path.join('mallm-v2', f'{i}.mp3')
    align = os.path.join('mallm-v2-alignment_alignment', f'{i}.json')
    
    if not os.path.exists(audio):
        continue
    
    with open(align) as fopen:
        align = json.load(fopen)
    
    merged_data = []
    for entry in align[1:]:
        if entry['text'] == '.' and merged_data:
            merged_data[-1]['text'] += '.'
            merged_data[-1]['end'] = entry['end']
        else:
            merged_data.append(entry)

    for i in range(len(merged_data)):
        merged_data[i]['text'] = replace.get(merged_data[i]['text'], merged_data[i]['text'])
        
    ts = []
    for c_ in merged_data:
        start = min(timestamps, key=lambda t: abs(t - (c_['start'])))
        end = min(timestamps, key=lambda t: abs(t - (c_['end'])))
        w = c_['text']
        t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
        ts.append(t)

    ts = ''.join(ts)
    new_text = text = f"<|startoftranscript|><|ms|><|transcribeprecise|>{ts}<|endoftext|>"

    result.append({
        'new_text': new_text,
        'audio_filename': audio,
    })

In [4]:
len(result)

2422

In [9]:
!zip -rq mallm-v2.zip mallm-v2

In [10]:
import pandas as pd

pd.DataFrame(result).to_parquet('mallm-word-timestamps.parquet')

In [11]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="mallm-word-timestamps.parquet",
    path_in_repo="data/mallm_word-00000-of-00001.parquet",
    repo_id="malaysia-ai/STT-Whisper",
    repo_type="dataset",
)

mallm-word-timestamps.parquet:   0%|          | 0.00/846k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/STT-Whisper/commit/1fef0f41dd14ec04202b4ffe819427ea5510d065', commit_message='Upload data/mallm_word-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='1fef0f41dd14ec04202b4ffe819427ea5510d065', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/malaysia-ai/STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='malaysia-ai/STT-Whisper'), pr_revision=None, pr_num=None)

In [12]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="mallm-v2.zip",
    path_in_repo="mallm-v2.zip",
    repo_id="malaysia-ai/STT-Whisper",
    repo_type="dataset",
)

mallm-v2.zip:   0%|          | 0.00/234M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/STT-Whisper/commit/3366c6e13701299d1666dd0575a7a52969b53d44', commit_message='Upload mallm-v2.zip with huggingface_hub', commit_description='', oid='3366c6e13701299d1666dd0575a7a52969b53d44', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/malaysia-ai/STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='malaysia-ai/STT-Whisper'), pr_revision=None, pr_num=None)